In [32]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms.v2 import Compose
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights

from video_dataset import VideoDataset
from train_model import TrainModel

BATCH_SIZE = 4
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {DEVICE}")

In [19]:
model = r2plus1d_18(weights=R2Plus1D_18_Weights.DEFAULT)

In [28]:
# Replace head

model.fc = nn.Linear(model.fc.in_features, 2)

# Freeze all layers except the new head
for key, params in model.named_parameters():
    if 'fc' not in key:
        params.requires_grad = False


print(model) # Perhaps we should also print torchinfo.summary here

VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [33]:
# Initialize train and validation datasets
# TODO videodataset root paths and transforms - this depends on Victor's offline preprocessing 
train_root = ''
train_transform = Compose([])

train_ds = VideoDataset(root_path=train_root, transform=train_transform)

validation_root = ''
validation_transform = Compose([])

validation_ds = VideoDataset(root_path=validation_root, transform=validation_transform)

# Initialize dataloaders

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
validation_dl = DataLoader(validation_ds, batch_size=BATCH_SIZE, shuffle=False)

ValueError: Pass at least one transform

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # TODO arguments for the optimizer
loss_fn = nn.CrossEntropyLoss() # TODO
score_fn = nn.CrossEntropyLoss() # TODO

In [ ]:
# Train the model!

model, train_loss, train_score, validation_loss, validation_score, _ = TrainModel(
  model, 
  train_dl, 
  validation_dl, 
  optimizer, 
  NUM_EPOCHS, 
  loss_fn, 
  score_fn
)

In [ ]:
# TODO plot the results?